In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from scholarly import scholarly 
import urllib.request
from PIL import Image
import os
import csv
import ast

In [2]:
df = pd.read_csv('Assignment1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'Assignment1.csv'

In [ ]:
df['DR-NTU URL']

In [ ]:
# Getting the profile pictures of the professors

os.chdir("dp")

count = 0
for url in df['DR-NTU URL']:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    src = soup.find(class_ = 'image').find('img')['src']
    src = src.replace(' ', '%20')
    img_src = 'https://dr.ntu.edu.sg' + src
    img = Image.open(requests.get(img_src,stream=True).raw)
    img = img.convert('RGB')
    img.save(str(count)+ '.jpg')
    count+=1

In [ ]:
# Getting the background / biography of the professors

bio = []
for url in df['DR-NTU URL']:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    c = soup.find(id = 'biographyDiv').text.replace('\n', '')
    bio.append(c)

bio_df = pd.DataFrame(bio, columns = ['Biography'])

In [ ]:
bio_df.to_csv('bio.csv', index = False)

In [ ]:
research_interest = []
profile_pic = []
prof_name = []
for names in df['Full Name']:
    if names == "Ke Yiping, Kelly" :
        names = names.replace("Ke Yiping, Kelly", "Ke Yiping")
    names = names + ', Nanyang Technological University'
    search_query = scholarly.search_author(names)
    try:
        author = next(search_query)
        if len(author['interests']) == 0:
            research_interest.append(None)
        else:
            research_interest.append(author['interests'])
        profile_pic.append(author['url_picture'])
        prof_name.append(author['name'])
        
    except StopIteration:
        research_interest.append(None)
        profile_pic.append(None)
        prof_name.append(None)

new_df = pd.DataFrame({'Name': prof_name, 'Research Interest': research_interest, 'Profile Picture': profile_pic, })

In [ ]:
test = pd.read_csv('test.csv')
df['Research Interest'] = test['Research Interest']
df

In [ ]:
keywords_df = df[df['Research Interest'].isna()]
keywords_df

In [ ]:
research_interest = []
for url in keywords_df['DR-NTU URL']:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    keyword_list = soup.find_all(class_ = 'rkeyword')
    keywords = []
    for item in keyword_list:
        c = item.text.strip()
        keywords.append(c)
    research_interest.append(keywords)
    

In [ ]:
df['Research Interest'].fillna(research_interest, inplace = True)

In [ ]:
# Getting co-authors from dblp
os.chdir("co_author")


count = 0
for url in df['DBLP URL']:
    names = []
    href = []
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        co_author = soup.find_all(class_ = 'person')
        for item in co_author:
            names.append(item.text)
            href.append(item.find('a')['href'])
        pd.DataFrame(names,href).to_csv(str(count) + '.csv', header = False)
    except:
        print('No dblp page found for ' + str(url))
    count+=1

    

In [ ]:
# trying to get the journals

soup.find(id='coauthor-section').find_all('div')[9::]

In [ ]:
bio = pd.read_csv('bio.csv')


In [ ]:
df = pd.read_csv('Assignment2.csv')
df['Biography'] = bio['Biography']

In [ ]:
df.to_csv('Assignment2.csv', index = False)

In [ ]:
for index,row in df.iterrows():
    image_path = "dp/" + str(index) + ".jpg"
    print(image_path)
Image.open(image_path)

In [ ]:
for index,row in df.iterrows():
    c = row['Research Interest']
    print(c)

In [ ]:
df['Research Interest'] = df['Research Interest'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
df['Research Interest'][0][0]

In [ ]:
from scholarly import scholarly

search_query = scholarly.search_author('AS Madhukumar')
author = next(search_query)
c = scholarly.fill(author,sections = ['basics','counts'])

In [ ]:
savethis = c['cites_per_year']

df = pd.DataFrame(list(savethis.items()), columns=['Year', 'Citations'])
df

In [ ]:
# Searching for prof's names on Google Scholar with NTU tag using Scholarly
test_name = []
test_citation = []
test_affiliation = []

citations = []
for names in df['Full Name']:
    if names == "Ke Yiping, Kelly" :
        names = names.replace("Ke Yiping, Kelly", "Ke Yiping")
    if names == 'Li Fang':
        continue
    names = names + ', Nanyang Technological University'
    search_query = scholarly.search_author(names)
    try:
        author = next(search_query)
        
        # Test DataFrame to check if the names and affiliation from Scholarly is correct.
        test_name.append(author['name'])
        test_citation.append(author['citedby'])
        test_affiliation.append(author['affiliation'])

        # Append the citations to the list
        citations.append(author['citedby'])
    except StopIteration:
        citations.append(None)
        # print(f"Professor {names} not found")

In [ ]:
df = pd.read_csv('Assignment2.csv')

os.chdir("citations")
for names in df['Full Name']:
    if names == "Ke Yiping, Kelly" :
        names = names.replace("Ke Yiping, Kelly", "Ke Yiping")
    if names == 'Li Fang':
        continue
    try:
        names = names + ', Nanyang Technological University'
        search_query = scholarly.search_author(names)
        author = next(search_query)
        c = scholarly.fill(author,sections = ['basics','counts'])
        savethis = c['cites_per_year']
        df = pd.DataFrame(list(savethis.items()), columns=['Year', 'Citations'])
        df.to_csv(names + '.csv', index = False)
    except:
        print(f"Professor {names} not found")
        
        


In [ ]:
df = pd.read_csv("Assignment2.csv")
# df = df.iloc[0]
df['Research Interest'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [ ]:
df['Research Interest']

In [ ]:
index = df.loc[df['Full Name'] == 'Bo An'].index[0]

In [ ]:
index

In [ ]:
import networkx as nx

G = nx.Graph()



df = pd.read_csv('co_author/0.csv', header = None)
df

In [ ]:
df1 = pd.read_csv('Assignment2.csv')
for i in range(86):
    if i == 67:
        continue
    df = pd.read_csv('co_author/' + str(i) + '.csv', header = None)
    for index1,row1 in df1.iterrows():
        # print(row1['Full Name'])
        G.add_node(row1['Full Name'])
        for index,row in df.iterrows():
            # print(row[1])
            G.add_node(row[1])
            # print(row1["Full Name"], row[1])
            G.add_edge(row1["Full Name"], row[1])

In [ ]:
df = pd.read_csv('Assignment2.csv')
scse_prof_names = df["Full Name"]

In [ ]:
df1 = pd.read_csv('co_author/0.csv', header = None)
co_author_names = df1[1]

In [ ]:
#SCSE Network
import networkx as nx

G = nx.Graph()


df1 = pd.read_csv('Assignment2.csv')
faculty_names = df1['Full Name'].tolist()

for name in faculty_names:
    G.add_node(name)

for i in range(86):
    if i == 67:
        continue
    co_author_df = pd.read_csv('co_author/' + str(i) + '.csv', header = None)
    current_prof = faculty_names[i]

    for _,row in co_author_df.iterrows():
        co_author_name = row[1]
        if co_author_name in faculty_names:
            G.add_edge(current_prof, co_author_name)

In [ ]:
nx.draw(G, with_labels=True, node_size=300, node_color='skyblue', font_size=10)

In [ ]:
import matplotlib.pyplot as plt

# Draw the graph
nx.draw(G, with_labels=True, node_size=300, node_color='skyblue', font_size=10)

# Display the graph
plt.show()


In [ ]:
# column_headers = [
#     "what",
#     "Title",
#     "Acronym",
#     "Source",
#     "Rank",
#     "Note",
#     "DBLP",
#     "Primary FoR",
#     "Comments",
#     "Average Rating"
# ]

# df = pd.read_csv("CORE.csv", names=column_headers,)
# df.drop(columns=["what","Note","DBLP","Primary FoR","Comments","Average Rating"], inplace=True)

In [ ]:
df = pd.read_csv("CORE.csv")
df

In [ ]:
df['Rank'].unique()

In [ ]:
prof = pd.read_csv("publications/Arvind_Easwaran.csv")
prof

In [ ]:
prof[prof['Journal Acronym'].isin(df['Acronym'].str.lower())]

In [ ]:
# Assuming df1 and df2 are your DataFrames
import pandas as pd

df=pd.read_csv("CORE.csv")
prof = pd.read_csv("publications\A_S_Madhukumar.csv")

# Convert col2 in df1 and col3 in df2 to lowercase (or uppercase)
# df['Acronym'] = df['Acronym'].str.lower()

# Create a boolean mask based on the condition
condition = prof['Journal Acronym'].isin(df['Acronym'])

# Use the boolean mask to filter rows in df1
result_df = prof[condition]
result_df


In [ ]:
ranking_df = df[df['Acronym'].isin(result_df['Journal Acronym'])]
ranking_df

In [ ]:
merged_df = pd.merge(result_df, ranking_df, left_on='Journal Acronym',right_on="Acronym" ,how='inner')
merged_df

In [9]:
df = pd.read_csv('Assignment2.csv')

os.chdir("indices")
for names in df['Full Name']:
    if names == "Ke Yiping, Kelly" :
        names = names.replace("Ke Yiping, Kelly", "Ke Yiping")
    if names == 'Li Fang':
        continue
    try:
        names = names + ', Nanyang Technological University'
        search_query = scholarly.search_author(names)
        author = next(search_query)
        c = scholarly.fill(author,sections = ['basics','indices'])
        hindex = c['hindex']
        hindex5y = c["hindex5y"]
        i10index = c['i10index']
        i10index5y = c['i10index5y']
        df = pd.DataFrame({"hindex":hindex, "hindex5y":hindex5y, "i10index":i10index, "i10index5y":i10index5y}, index = [0])
        names.split(',')[0]
        df.to_csv(names + '.csv', index = False)
    except:
        print(f"Professor {names} not found")
        
        


Professor Chan Syin, Nanyang Technological University not found
Professor Douglas Leslie Maskell, Nanyang Technological University not found
Professor Jagath Chandana Rajapakse, Nanyang Technological University not found
Professor Josephine Chong, Nanyang Technological University not found
Professor Joty Shafiq Rayhan, Nanyang Technological University not found
Professor Lau Chiew Tong, Nanyang Technological University not found
Professor Li Mo, Nanyang Technological University not found
Professor Luke Ong （翁之昊）, Nanyang Technological University not found
Professor Pan, Sinno Jialin, Nanyang Technological University not found
Professor Quek Hiok Chai, Nanyang Technological University not found
Professor Sourav Saha Bhowmick, Nanyang Technological University not found
Professor Tang Xueyan, Nanyang Technological University not found
Professor Tay Kian Boon, Nanyang Technological University not found
Professor Thambipillai Srikanthan, Nanyang Technological University not found
Professor 

In [ ]:
names = 'Bo An, Nanyang Technological University'
search_query = scholarly.search_author(names)
author = next(search_query)
c = scholarly.fill(author,sections = ['basics','indices'])
hindex = c['hindex']
hindex5y = c["hindex5y"]
i10index = c['i10index']
i10index5y = c['i10index5y']
df = pd.DataFrame({"hindex":hindex, "hindex5y":hindex5y, "i10index":i10index, "i10index5y":i10index5y}, index = [0])

In [ ]:
names.split(',')[0]


In [14]:
os.getcwd()

'e:\\Profound'

In [56]:
df = pd.read_csv("indices\A S Madhukumar, Nanyang Technological University.csv")

In [63]:
newdf = df.transpose()
newdf.columns = ['Total']
year_2018 = []
year_2018.append(newdf.iloc[1][0])
year_2018.append(newdf.iloc[3][0])
newdf.drop(["hindex5y","i10index5y"], inplace = True)
newdf['2018'] = year_2018
newdf

In [76]:
data = {'Column1': [1, 2, 3, 4],
        'Column2': ['A', 'B', 'C', 'D']}
df = pd.DataFrame(data)

# Get the index of the second row (row at index 1)
row_index = df.index[0]

row_index

0